## 01. Naver Blog Crawling (basic)
https://developers.naver.com/docs/search/blog/

#### 1. 개인 application ID, secret 발급받기 
#### 2. Python 활용시 블로그 검색 API 확인하기 

In [ ]:
import os
import sys
import urllib.request
naver_client_id = "S79oTS9dx6TONhrab83P" #네이버 어플리케이션 ID 입력
naver_client_secret = "bV5K1PSVDD"# 네이버 어플리케이션 SECRET 입력
encText = urllib.parse.quote("수트패션") # 검색할 단어 입력

url = "https://openapi.naver.com/v1/search/blog?query=" + encText # json 결과  #검색어 qeury의 검색할 때 사용하는 url
                                                                                                                  
    # url = "https://openapi.naver.com/v1/search/blog.xml?query=" + encText # xml 결과  
                                                                       
        
request = urllib.request.Request(url)      
request.add_header("X-Naver-Client-Id",naver_client_id)  
request.add_header("X-Naver-Client-Secret",naver_client_secret)

response = urllib.request.urlopen(request)  
rescode = response.getcode()              #요청(request)을 반환(response)하는 것

if(rescode==200):                        #서버가 요청을 제대로 처리했다는 뜻의 200
    response_body = response.read()      #response.read()함수는 요청한 페이지를 읽어들인다는 의미
    print(response_body.decode('utf-8'))
else:
    print("Error Code:" + rescode)

## 02. Naver Blog Crawling (Advanced)
https://developers.naver.com/docs/search/blog/

#### 1. BeautifulSoup 사용하기 : HTML, XML 등의 문서를 구문분석하기 위한 파이썬 라이브러리
#### 2. 블로그 수집하고, CSV파일로 저장하기 


In [ ]:
# 필요한 라이브러리 설치

import re               #정규표현식 사용을 위한 라이브러리
import json             #네이버API를 통해 가져온 JSON 파일을 처리하기 위한 라이브러리
import math             #수학 라이브러리
import requests         #http 통신을 사용
import urllib.request   #웹과 관련된 내용을 처리하는 라이브러리: urllib
import urllib.error     
import urllib.parse       ##파싱(parse): 데이터를 특정 패턴이나 순서로 추출해 가공하는 것>> 파서(parser)로 웹페이지 구조를 읽어들임
import pandas as pd

In [ ]:
# BeaurifulSoup 설치  : 네이버 API는 블로그 일부 내용만 반환하여,크롤링 라이브러리인 BeautifulSoup를 설치하고 전체 블로그 URL로 접속하여 포스트 내용 전체를 가져와 저장
!pip install bs4

In [ ]:
 from bs4 import BeautifulSoup

In [ ]:
naver_client_id = "S79oTS9dx6TONhrab83P" #네이버 어플리케이션 ID 입력
naver_client_secret = "bV5K1PSVDD" # 네이버 어플리케이션 SECRET 입력

In [ ]:
#블로그 개수 카운트
#블로그 전체 개수를 가져오는 함수: get_blog_count()
#query=검색 키워드

def get_blog_count(query, display):
    encode_query = urllib.parse.quote(query)
    search_url = "https://openapi.naver.com/v1/search/blog?query=" + encode_query
    request = urllib.request.Request(search_url)

    request.add_header("X-Naver-Client-Id", client_id)
    request.add_header("X-naver-Client-Secret", client_secret)
    
    response = urllib.request.urlopen(request)
    response_code = response.getcode()

    if response_code == 200:
        response_body = response.read()
        response_body_dict = json.loads(response_body.decode('utf-8'))

        print("Last build date: " + str(response_body_dict['lastBuildDate']))
        print("Total: " + str(response_body_dict['total']))
        print("Start: " + str(response_body_dict['start']))
        print("Display: " + str(response_body_dict['display']))

        if response_body_dict['total'] == 0:
            blog_count = 0
        else:
            blog_total = math.ceil(response_body_dict['total'] / int(display))

            if blog_total >= 1000:
                blog_count = 1000          #네이버에서는 검색에 대한 최대 1000개의 검색결과만 보여주기 때문에
            else:
                blog_count = blog_total

            print("Blog total: " + str(blog_total))
            print("Blog count: " + str(blog_count))
        
        return blog_count
    
    #(query, display) parameter 입력하여 get_blog_count 함수 테스트

In [ ]:
get_blog_count('패션', 10)

In [ ]:
# 블로그 내용 가져오는 코드

def get_blog_post(query, display, start_index, sort):
    global no, df
    
    encode_query = urllib.parse.quote(query)
    search_url = "https://openapi.naver.com/v1/search/blog?query=" + encode_query + "&display=" + str(display) + "&start=" + str(start_index) + "&sort=" + sort

    request = urllib.request.Request(search_url)

    request.add_header("X-Naver-Client-Id", naver_client_id)
    request.add_header("X-Naver-Client-Secret", naver_client_secret)

    response = urllib.request.urlopen(request)
    response_code = response.getcode()

    if response_code == 200:
        response_body = response.read()
        response_body_dict = json.loads(response_body.decode('utf-8'))
        for item_index in range(0, len(response_body_dict['items'])):
            try:
                remove_html_tag = re.compile('<.*?>')
                title = re.sub(remove_html_tag, '', response_body_dict['items'][item_index]['title'])
                link = response_body_dict['items'][item_index]['link'].replace("amp;", "")
                description = re.sub(remove_html_tag, '', response_body_dict['items'][item_index]['description'])
                blogger_name = response_body_dict['items'][item_index]['bloggername']
                blogger_link = response_body_dict['items'][item_index]['bloggerlink']
                post_date = response_body_dict['items'][item_index]['postdate']

                no += 1
                post_code = requests.get(link)
                post_text = post_code.text
                post_soup = BeautifulSoup(post_text, 'lxml')

                blog_post_content_text = ""
                for mainFrame in post_soup.select('iframe#mainFrame'):
                    blog_post_url = "http://blog.naver.com" + mainFrame.get('src')
                    blog_post_code = requests.get(blog_post_url)
                    blog_post_text = blog_post_code.text
                    blog_post_soup = BeautifulSoup(blog_post_text, 'lxml')
                    
                    for blog_post_content in blog_post_soup.find_all('div', class_='se-viewer'):
                        blog_post_content_text = blog_post_content.get_text()
                        
                    for blog_post_content in blog_post_soup.find_all('div', class_='se_doc_viewer'):
                        blog_post_content_text = blog_post_content.get_text()
 
                    for blog_post_content in blog_post_soup.select('div#postViewArea'):
                        blog_post_content_text = blog_post_content.get_text()

                df.loc[no] = [title, link, description, blogger_name, blogger_link, post_date, blog_post_content_text]
                print("#", end='')
                
            except:
                item_index += 1

In [ ]:
#(query, display, start_index, sort) 입력하여 get_blog_post 함수 테스트

get_blog_post('수트패션', 10, 1, 'sim')

In [ ]:
no = 0                 # 몇개의 포스트를 저장하였는지 세기 위한 index
query = '수트패션'              # 검색을 원하는 문자열로서 UTF-8로 인코딩한다.
display = 10            # 검색 결과 출력 건수 지정, 10(기본값),100(최대)
start = 1              # 검색 시작 위치로 최대 1000까지 가능
sort =  'sim'               

df = pd.DataFrame(columns=("Title", "Link", "Description", "Blogger Name", "Blogger Link", "Post Date", "Post Contents"))

blog_count = get_blog_count(query, display)
for start_index in range(start, blog_count + 1, display):  #for 함수는 반복함수 
    get_blog_post(query, display, start_index, sort)
    
df.to_csv("21_suit_fashion.csv".header=True, index=False)

In [ ]:
# csv 파일 불러와서 잘 저장되었는지 테스트

data = pd.reac_csv('21_suit_fashion.csv')
data.head()